<a href="https://colab.research.google.com/github/ujkim25/sparta_ai_engineering/blob/main/RAG_Vector_DB2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 27.8 MB/s eta 0:00:00


In [2]:
import requests
from io import StringIO
import pandas as pd
import faiss
from sentence_transformers import SentenceTransformer

In [3]:
res = requests.get('https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/sick2014/SICK_train.txt')

data = pd.read_csv(StringIO(res.text), sep='\t')
data.head()

,pair_ID,sentence_A,sentence_B,relatedness_score,entailment_judgment
0,1,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,4.5,NEUTRAL
1,2,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,3.2,NEUTRAL
2,3,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,4.7,ENTAILMENT
3,5,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,3.4,NEUTRAL
4,9,The young boys are playing outdoors and the ma...,A group of kids is playing in a yard and an ol...,3.7,NEUTRAL


In [4]:
sentences = data["sentence_A"].tolist()
print(len(sentences))

4500


In [5]:
urls = [
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/MSRpar.train.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/MSRpar.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2013/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2014/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2014/images.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2015/images.test.tsv'
]

In [6]:
for url in urls:
    res = requests.get(url)
    # extract to dataframe
    data = pd.read_csv(StringIO(res.text), sep='\t', header=None, on_bad_lines="skip")
    # add to columns 1 and 2 to sentences list
    sentences.extend(data[1].tolist())
    sentences.extend(data[2].tolist())

In [7]:
sentences = [word for word in list(set(sentences)) if type(word) is str]
len(sentences)

12848

In [9]:
from transformers import AutoTokenizer, AutoModel

In [10]:
tokenizer = AutoTokenizer.from_pretrained("snumin44/simcse-ko-roberta-unsupervised")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/971 [00:00<?, ?B/s]

In [11]:
model = AutoModel.from_pretrained("snumin44/simcse-ko-roberta-unsupervised")

config.json:   0%|          | 0.00/744 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

In [21]:
input_sentence = "궁극적으로 유사도를 검색해서 top5를 가져와서 LLM에게 쿼리를 하면 일관성이나 정확성이 담보되는 건가요?"
sentence_A = "궁극적으로 가나 유사도를 가나 검색해서 가나 top5를 가나 가져와서 가나 LLM에게 가나 쿼리를 하면 일관성이나 정확성이 담보되는 건가요?"
sentence_B = "아 어제 cuda로 gpu 올렸는데 cpu나온게 faiss-cpu 인스톨해서 그런건가요?"

In [13]:
tokenized_output = tokenizer(input_sentence)

In [14]:
tokenized_output["input_ids"]

[0,
 8225,
 11187,
 5670,
 2119,
 2138,
 6102,
 6396,
 7052,
 2006,
 2049,
 2138,
 17852,
 2112,
 48,
 2237,
 2110,
 2170,
 2318,
 1743,
 2059,
 2138,
 1889,
 2460,
 7182,
 11604,
 2075,
 4480,
 11604,
 6484,
 2496,
 2259,
 12039,
 35,
 2]

In [15]:
print( [tokenizer.convert_ids_to_tokens(token_id) for token_id in  tokenized_output["input_ids"]] )

['[CLS]', '궁극', '##적으로', '유사', '##도', '##를', '검색', '##해서', 'to', '##p', '##5', '##를', '가져와', '##서', 'L', '##L', '##M', '##에', '##게', '쿼', '##리', '##를', '하', '##면', '일관', '##성이', '##나', '정확', '##성이', '담보', '##되', '##는', '건가요', '?', '[SEP]']


In [16]:
model

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(32000, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dr

In [17]:
tokenized_output = tokenizer(input_sentence, return_tensors = "pt")

In [18]:
tokenized_output

{'input_ids': tensor([[    0,  8225, 11187,  5670,  2119,  2138,  6102,  6396,  7052,  2006,
          2049,  2138, 17852,  2112,    48,  2237,  2110,  2170,  2318,  1743,
          2059,  2138,  1889,  2460,  7182, 11604,  2075,  4480, 11604,  6484,
          2496,  2259, 12039,    35,     2]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [26]:
output_1 = model(input_ids = tokenized_output["input_ids"], attention_mask = tokenized_output["attention_mask"], token_type_ids = tokenized_output["token_type_ids"])

In [20]:
output.pooler_output[0]

tensor([ 4.2522e-01, -4.4108e-02, -2.6776e-01, -2.5126e-01, -9.8526e-02,
         4.8765e-02, -2.3226e-01, -1.1906e-02,  4.9962e-01,  8.2058e-02,
         1.2707e-01, -9.0872e-02,  1.8105e-01, -4.1344e-01,  6.6924e-02,
        -2.5707e-01, -1.4769e-01,  1.6417e-01,  1.7021e-01, -3.8228e-01,
         2.1732e-01,  3.3491e-01,  3.1895e-02, -1.9710e-01,  9.1979e-02,
         4.4820e-01, -1.3145e-01,  1.6133e-01, -3.6375e-01,  8.4189e-02,
        -9.5511e-02,  1.9693e-01, -4.2704e-01, -1.7407e-02, -2.1583e-01,
         3.0397e-01, -3.9552e-02, -6.7853e-01,  7.9501e-02,  4.6252e-02,
        -1.2198e-01, -1.8017e-01, -1.0439e-02,  2.2344e-01,  7.6353e-02,
         3.3020e-01, -7.0137e-02, -4.3343e-01, -4.9718e-01,  2.1821e-01,
        -3.8262e-01, -3.0990e-01,  1.7769e-01, -1.1137e-01,  3.6263e-01,
         1.5040e-01, -3.8995e-01, -3.0913e-01, -4.0129e-02, -1.1999e-02,
        -2.2643e-01,  1.9949e-01,  3.6262e-01,  1.1087e-01,  4.7484e-01,
        -7.1861e-02, -2.2133e-01, -7.1335e-02, -1.4

In [23]:
tokenized_output_A = tokenizer(sentence_A, return_tensors="pt")
tokenized_output_B = tokenizer(sentence_B, return_tensors="pt")

In [24]:
output_A = model(**tokenized_output_A)
output_B = model(**tokenized_output_B)

In [27]:
output_1_embedding = output_1.pooler_output[0]
output_A_embedding = output_A.pooler_output[0]
output_B_embedding = output_B.pooler_output[0]

In [28]:
output_1_embedding[:5]

tensor([ 0.4252, -0.0441, -0.2678, -0.2513, -0.0985], grad_fn=<SliceBackward0>)

In [29]:
output_A_embedding[:5]

tensor([ 2.1208e-01,  8.2523e-05, -2.4556e-01, -1.3020e-01, -6.4491e-02],
       grad_fn=<SliceBackward0>)

In [30]:
output_B_embedding[:5]

tensor([-0.0391, -0.3279,  0.1013, -0.3819,  0.0833], grad_fn=<SliceBackward0>)

In [31]:
from sentence_transformers import util

In [32]:
A = util.pytorch_cos_sim(output_1_embedding, output_A_embedding)
B = util.pytorch_cos_sim(output_1_embedding, output_B_embedding)

In [33]:
A

tensor([[0.7469]], grad_fn=<MmBackward0>)

In [34]:
B

tensor([[-0.0485]], grad_fn=<MmBackward0>)

In [35]:
model = SentenceTransformer("bert-base-nli-mean-tokens")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [47]:
sentence_embeddings = model.encode(sentences)

KeyboardInterrupt: 

In [ ]:
sentence_embeddings = model.encode(sentences)
dim = sentence_embeddings.shape[1]
print(dim)
nlist = 50
quantizer = faiss.IndexFlatL2(dim)
index = faiss.IndexIVFFlat(quantizer, dim, nlist)
index.is_trained
index.train(sentence_embeddings)
index.is_trained
index.add(sentence_embeddings)
index.ntotal

k = 4
xq = model.encode(["Someone sprints with a football"])
xq.shape

In [38]:
search_results = [
    "That was up from the year-ago quarter, when the company earned $54.3 million, or 22 cents a share.",
    'It added it had "co-operated fully" with police since November.',
    'A child, who is small, is outdoors climbing steps outdoors in an area full of grass',
    "(cause to) move gently back and forth"
]

In [39]:
context_list = []
for i, result in enumerate(search_results):
    context_list.append(
        f"[문서] {i+1}, [내용]: {result}"
    )

In [43]:
from openai import OpenAI

In [44]:
client=OpenAI(api_key="")

In [48]:
messages = [
    {
        "role":"user",
        "content": f"다음 문서들을 참고해서 질문에 답변해주세요: \n\n [참고문서]{context} [질문] Someone sprints with a football"
    }
]

NameError: name 'context' is not defined

In [49]:
response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        temperature=0.3
    )

NameError: name 'messages' is not defined

In [50]:
response.choices[0].message.content

NameError: name 'response' is not defined